<a href="https://colab.research.google.com/github/centeno/covid-impact-se/blob/master/step01_1_collection_grey_GoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [784]:

#!apt-get update # to update ubuntu to correctly run apt install

'''

# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3


'''

'\n\n# scraper\n!pip install selenium\n!apt install chromium-chromedriver\n!pip install webdriver-manager\n!cp /usr/lib/chromium-browser/chromedriver/usr/bin\n\n!apt-get update\n!apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\n!pip install selenium msticpy tld folium\n!pip pymongo pymongo[srv] dnspython\n\n# geocoder ipaddress\n!pip install msticpy\n\n# get domain\n!pip3 install tld\n\n\n# spacy\n!pip install -U pip setuptools wheel\n!pip install -U spacy\n#!python -m spacy download en\n#!python -m spacy download pt\n!python -m spacy download en_core_web_sm\n# spacy contextual Spell Check\n!pip install contextualSpellCheck\n!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3\n\n\n'

### Import das dependencias

In [785]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [786]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [787]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/01_grey/2021/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2021-01-01.csv	2021-01-19.csv	2021-02-06.csv	2021-02-24.csv	2021-03-14.csv
2021-01-02.csv	2021-01-20.csv	2021-02-07.csv	2021-02-25.csv	2021-03-15.csv
2021-01-03.csv	2021-01-21.csv	2021-02-08.csv	2021-02-26.csv	2021-03-16.csv
2021-01-04.csv	2021-01-22.csv	2021-02-09.csv	2021-02-27.csv	2021-03-17.csv
2021-01-05.csv	2021-01-23.csv	2021-02-10.csv	2021-02-28.csv	2021-03-18.csv
2021-01-06.csv	2021-01-24.csv	2021-02-11.csv	2021-03-01.csv	2021-03-19.csv
2021-01-07.csv	2021-01-25.csv	2021-02-12.csv	2021-03-02.csv	2021-03-20.csv
2021-01-08.csv	2021-01-26.csv	2021-02-13.csv	2021-03-03.csv	2021-03-21.csv
2021-01-09.csv	2021-01-27.csv	2021-02-14.csv	2021-03-04.csv	2021-03-22.csv
2

### Import chaves de api

In [788]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [789]:
import pandas as pd
GSE_RESULTS_CLASS = 'ZINbbc xpd O9g5cc uUPGi'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  d_before = (date + DateOffset(days = 2)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = " + "after:" + date.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', '').split('&sa')[0],
         
    }
    
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [790]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

# Scraper 3 (2021 v2)
# keyword = '"covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work"'

### Merge csv's or SCRAPER new search

In [791]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])
else:
  #get first date
  first_date = date(2021, 1, 1)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df = fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
100,100,2021-03-31,31 Crucial Insider Threat Statistics: 2021 Lat...,Frequency of attacks · 60% of data breaches ar...,https://financesonline.com/insider-threat-stat...,https,financesonline.com,/insider-threat-statistics/,NaN,NaN,financesonline.com,192.124.249.111,US,United States,California,Menifee,-117.189110,33.657532,30148.0,NaN,geolocation,{},192.124.249.111,31 Crucial Insider Threat Statistics: 2021 Lat...,Frequency of attacks · 60% of data breaches ar...
101,101,2021-03-31,[PDF] Annual rEPORT 2020 - LUMS Consultancy Group,"Apr 1, 2021 · Leveraging drone technology in a...",https://lcg.lums.edu.pk/wp-content/uploads/202...,https,lcg.lums.edu.pk,/wp-content/uploads/2021/04/Report-1.pdf,NaN,NaN,lums.edu.pk,203.135.63.169,PK,Pakistan,Punjab,Lahore,74.313332,31.563330,17557.0,NaN,geolocation,{},203.135.63.169,[PDF] Annual rEPORT 2020 - LUMS Consultancy Group,"Apr 1, 2021 · Leveraging drone technology in a..."
102,102,2021-03-31,[PDF] Download the 2020 Annual Report - RSA Group,"Apr 1, 2021 · customers, operational resilienc...",https://www.rsagroup.com/media/kfwfyt1d/rsa-an...,https,www.rsagroup.com,/media/kfwfyt1d/rsa-annual-report-and-accounts...,NaN,NaN,rsagroup.com,52.157.222.206,NL,Netherlands,North Holland,Diemen,4.940190,52.309052,8075.0,NaN,geolocation,{},52.157.222.206,[PDF] Download the 2020 Annual Report - RSA Group,"Apr 1, 2021 · customers, operational resilienc..."
103,103,2021-03-31,[PDF] A Taxonomy of UK National Laboratories -...,"context of its plans for national resilience, ...",https://www.airto.co.uk/wp-content/uploads/202...,https,www.airto.co.uk,/wp-content/uploads/2021/03/MASTER_AIRTO_PRESE...,NaN,NaN,airto.co.uk,206.189.24.22,GB,United Kingdom,England,Romford,0.116600,51.583302,14061.0,NaN,geolocation,{},206.189.24.22,[PDF] A Taxonomy of UK National Laboratories -...,"context of its plans for national resilience, ..."
104,104,2021-03-31,[PDF] Management-Matters-April-2021.pdf - LIBA,"Apr 1, 2021 · Social capital assumes a lot of ...",https://liba.edu/wp-content/uploads/2021/04/Ma...,https,liba.edu,/wp-content/uploads/2021/04/Management-Matters...,NaN,NaN,liba.edu,148.72.219.251,US,United States,Arizona,Scottsdale,-111.890106,33.610958,26496.0,NaN,geolocation,{},148.72.219.251,[PDF] Management-Matters-April-2021.pdf - LIBA,"Apr 1, 2021 · Social capital assumes a lot of ..."


### SCRAPER new search

In [792]:
#get next data
next_date = next_data(df)

#get seach
df_new = search(keyword, next_date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

#save df new
save(df_new)

#print new df
df_new.head(2)



https://www.google.com/search?q="covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working" after:2021-04-01 and before:2021-04-03&sort=date:r:2021-04-01:2021-04-03&num=200
horizon = after:2021-04-01 and before:2021-04-03


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-04-01,3 Digital Priorities for Software Engineering ...,"Apr 2, 2021 · On a daily basis, software engin...",https://www.gartner.com/smarterwithgartner/3-d...,https,www.gartner.com,/smarterwithgartner/3-digital-priorities-for-s...,,,gartner.com,65.8.181.123,US,United States,Florida,Flagami,-80.311958,25.762859,16509.0,None,geolocation,{},65.8.181.123,3 Digital Priorities for Software Engineering ...,"Apr 2, 2021 · On a daily basis, software engin..."
1,2021-04-01,Strickland Solutions | Global Leaders in Techn...,"Decades of business IT solutions, IT architect...",https://stricklandsolutions.com/,https,stricklandsolutions.com,/,,,stricklandsolutions.com,64.111.100.225,US,United States,California,Brea,-117.885399,33.927010,26347.0,None,geolocation,{},64.111.100.225,Strickland Solutions | Global Leaders in Techn...,"Decades of business IT solutions, IT architect..."
